# Classification des cellules sanguines cancéreuses (PyTorch).

## 1️⃣ Importation et préparation des données

### Imports : 

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from PIL import Image
import random
import os
import cv2


### Charger les images du dataset et vérifier leurs extensions (jpeg, jpg, bmp, png). Supprimer celles qui ne correspondent pas.

In [29]:
dataset_path = Path("../data/Blood_Cells_Cancer/")

valid_extensions = (".jpg", ".jpeg", ".png", ".bmp")

images = []
labels = []

In [30]:
problem_files = 0

for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    
    if os.path.isdir(class_path):
        for file_name in os.listdir(class_path):
            file_path = os.path.join(class_path, file_name)
            
            if file_name.lower().endswith(valid_extensions):
                
                try:
                    img = cv2.imread(file_path)
                    if img is not None:
                        images.append(img)
                        labels.append(class_name)
                    else:
                        problem_files += 1
                        print(f"Impossible de lire l'image : {file_path}")
                        # supprimer si le file ne s'ouvre pas
                        try:
                            os.remove(file_path)
                        except Exception as e:
                            print(f"Erreur lors de la suppression de {file_path} : {e}")
                        
                except Exception as e :
                    print(f"Erreur lors du chargement de {file_path} : {e}")
                    
            else:
                problem_files += 1
                print(f"Extension non valide, fichier {file_path}")
                # supprimer si le fichier n'a pas l'extension valide
                try:
                    os.remove(file_path)
                except Exception as e:
                    print(f"Erreur lors de la suppression de {file_path} : {e}")
                
        

In [31]:
print(f"Nombre total d'images valides : {len(images)}")
print(f"Nombre de fichiers problématiques : {problem_files}")

Nombre total d'images valides : 3242
Nombre de fichiers problématiques : 0
